# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data
### NOTE: HVPlot does not currently function, and Gmaps currently has a security problem with Jupyter Notebook. For this reason, instruction team recommended use of Plotly instead.

In [20]:
# Dependencies and Setup
import pandas as pd
#import hvplot.pandas # THIS DOES NOT FUNCTION, AND INSTRUCTION TEAM HAS SUGGESTED USING PLOTLY INSTEAD
import requests
import plotly.express as px


# Import API key
from api_keys import geoapify_key
#print(geoapify_key)

In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,10.66,66,100,3.13,AR,1667491311
1,1,rikitea,-23.1203,-134.9692,20.73,69,100,6.80,PF,1667491513
2,2,sibu,2.3000,111.8167,23.93,98,99,0.84,MY,1667491567
3,3,tasiilaq,65.6145,-37.6368,1.09,98,100,5.59,GL,1667491568
4,4,chaplynka,46.3651,33.5403,10.33,51,11,2.22,UA,1667491568


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
#%%capture --no-display

# Configure the map plot
fig = px.scatter_mapbox(city_data_df, lat="Lat", lon="Lng", hover_name="City", color_discrete_sequence=["fuchsia"], zoom=2, height=800, size="Humidity")
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})

# Display the map

fig.show()


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,10.66,66,100,3.13,AR,1667491311
1,1,rikitea,-23.1203,-134.9692,20.73,69,100,6.80,PF,1667491513
2,2,sibu,2.3000,111.8167,23.93,98,99,0.84,MY,1667491567
3,3,tasiilaq,65.6145,-37.6368,1.09,98,100,5.59,GL,1667491568
4,4,chaplynka,46.3651,33.5403,10.33,51,11,2.22,UA,1667491568


In [24]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"]< 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_weather.dropna()

# Display sample data
ideal_weather.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,coihaique,-45.5752,-72.0662,26.34,49,0,3.09,CL,1667491581
68,68,makakilo city,21.3469,-158.0858,25.01,79,0,2.57,US,1667491587
143,143,sakakah,29.9697,40.2064,21.89,36,0,1.79,SA,1667491616
152,152,wardha,20.7500,78.6167,21.27,56,0,0.89,IN,1667491619
236,236,bin qirdan,33.1378,11.2197,23.96,41,0,3.36,TN,1667491649


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
44,44,coihaique,-45.5752,-72.0662,26.34,49,0,3.09,CL,1667491581,
68,68,makakilo city,21.3469,-158.0858,25.01,79,0,2.57,US,1667491587,
143,143,sakakah,29.9697,40.2064,21.89,36,0,1.79,SA,1667491616,
152,152,wardha,20.7500,78.6167,21.27,56,0,0.89,IN,1667491619,
236,236,bin qirdan,33.1378,11.2197,23.96,41,0,3.36,TN,1667491649,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [41]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = f"No hotel found {response_address}"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
coihaique - nearest hotel: No hotel found <Response [200]>
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
sakakah - nearest hotel: sakakah residence
wardha - nearest hotel: Hotel Mina Palace
bin qirdan - nearest hotel: No hotel found <Response [200]>
tatawin - nearest hotel: No hotel found <Response [200]>
tuxpan - nearest hotel: Hotel Ruiz
kurud - nearest hotel: No hotel found <Response [200]>
puerto rico - nearest hotel: Suizo
camargo - nearest hotel: Hotel Santa Fe
dadri - nearest hotel: No hotel found <Response [200]>
karratha - nearest hotel: Karratha International Hotel
san vicente - nearest hotel: Los Lapachos
nanakuli - nearest hotel: Aulani, A Disney Resort & Spa
benghazi - nearest hotel: ترجاب
huntsville - nearest hotel: Embassy Suites
conway - nearest hotel: Fairfield Inn & Suites by Marriott Conway
sambava - nearest hotel: Royal Sava Hôtel
port hedland - nearest hotel: The Esplanade Hotel
comodoro rivadavia - nearest hotel: No hot

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
44,44,coihaique,-45.5752,-72.0662,26.34,49,0,3.09,CL,1667491581,No hotel found <Response [200]>
68,68,makakilo city,21.3469,-158.0858,25.01,79,0,2.57,US,1667491587,Embassy Suites by Hilton Oahu Kapolei
143,143,sakakah,29.9697,40.2064,21.89,36,0,1.79,SA,1667491616,sakakah residence
152,152,wardha,20.7500,78.6167,21.27,56,0,0.89,IN,1667491619,Hotel Mina Palace
236,236,bin qirdan,33.1378,11.2197,23.96,41,0,3.36,TN,1667491649,No hotel found <Response [200]>
238,238,tatawin,32.9297,10.4518,23.47,27,0,1.91,TN,1667491649,No hotel found <Response [200]>
253,253,tuxpan,19.5500,-103.4000,24.29,59,0,0.95,MX,1667491654,Hotel Ruiz
340,340,kurud,20.8322,81.7214,22.99,63,0,2.18,IN,1667491682,No hotel found <Response [200]>
341,341,puerto rico,-26.7960,-55.0240,26.70,33,0,0.20,AR,1667491682,Suizo
360,360,camargo,27.6667,-105.1667,21.63,23,0,2.51,MX,1667491691,Hotel Santa Fe


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
#%%capture --no-display

# Configure the map plot
fig = px.scatter_mapbox(hotel_df, lat="Lat", lon="Lng", hover_name="City", color_discrete_sequence=["fuchsia"], zoom=2, height=800, size="Humidity")
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})

# Display the map
fig.show()